
# Project 2 Validation Notebook

You cannot write any code in this notebook – to actually work on the project, open `project.py` and `project.ipynb`.

The only purpose of this notebook is to give you a blank copy of `project.ipynb`,
so that you can go to **Kernel > Restart & Run All** and ensure that all public `grader.check` cells pass using just the code in your `project.py`.

**Before submitting Project 2, make sure that the call to `grader.check_all()` at the bottom of this notebook shows that all test cases passed!** 
If it doesn't, there's likely a function in `project.ipynb` that is not implemented correctly in `project.py`, or it could be that a function in `project.py` depends on an object (e.g. a DataFrame) that is not an argument to that function.
    

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("project.ipynb")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import pathlib
from pathlib import Path

from IPython.display import clear_output

import urllib.request

In [ ]:
from project import *

In [ ]:
from ipywidgets import widgets, interact, FloatSlider
import plotly.express as px

In [ ]:
# data_dir will be use for multiple cell
data_dir = Path('data')
vacs_path = data_dir / 'covid_vaccinations.csv'
vacs = pd.read_csv(vacs_path)
vacs

In [ ]:
DEFAULT = 'US'

def plot_cases(country):
    country_only = vacs[vacs['Country_Region'] == country]
    fig = px.line(country_only, x='Date', y='Doses_admin', title=f"'Doses_admin' column for {country}")
    fig.show()
    

dropdown_cases = widgets.Dropdown(options=np.sort(vacs['Country_Region'].unique()), value=DEFAULT)

def dropdown_cases_handler(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        clear_output()
        display(dropdown_cases)
        plot_cases(change['new'])
        
display(dropdown_cases)
plot_cases(DEFAULT)
dropdown_cases.observe(dropdown_cases_handler)

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
mono = monotonic_violations_by_country(vacs)
mono

In [ ]:
grader.check("q1")

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
tots = robust_totals(vacs)
tots

In [ ]:
grader.check("q2")

In [ ]:
pops_raw = pd.read_csv(data_dir / 'populations.csv')
pops_raw

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
pops = fix_dtypes(pops_raw)
pops

In [ ]:
grader.check("q3")

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
missing = missing_in_pops(tots, pops)
missing

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
pops_fixed = fix_names(pops)
pops_fixed

In [ ]:
grader.check("q4")

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
fig = draw_choropleth(tots, pops_fixed)
fig.show()

In [ ]:
grader.check("q5")

In [ ]:
# Download Israeli COVID vaccinations data from the ☁️
if not pathlib.Path(data_dir / 'israel.csv').exists():
    urllib.request.urlretrieve(
        'https://f000.backblazeb2.com/file/dsc-data/covid-israel/israel.csv',
        data_dir / 'israel.csv'
    )

In [ ]:
israel_raw = pd.read_csv(data_dir / 'israel.csv')
israel_raw

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
israel = clean_israel_data(israel_raw)
israel

In [ ]:
grader.check("q6")

In [ ]:
israel_raw.info()

In [ ]:
israel.info()

In [ ]:
israel.assign(null_age=israel['Age'].isna()).groupby('null_age').mean()

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
# expect this cell to take a while to run!
test_stats_vax, test_stats_sick = mcar_permutation_tests(israel, 100)

In [ ]:
grader.check("q7")

In [ ]:
vax = israel.dropna()

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_overall = effectiveness(vax)
eff_overall

In [ ]:
grader.check("q8")

In [ ]:
AGE_GROUPS = [
    '12-15',
    '16-19',
    '20-29',
    '30-39',
    '40-49',
    '50-59',
    '60-69',
    '70-79',
    '80-89',
    '90-'
]

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
s_eff = stratified_effectiveness(vax)
s_eff

In [ ]:
grader.check("q9")

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_example = effectiveness_calculator(
    young_vaccinated_prop=0.01,
    old_vaccinated_prop=0.99,
    young_risk_vaccinated=0.01,
    young_risk_unvaccinated=0.20,
    old_risk_vaccinated=0.10,
    old_risk_unvaccinated=0.50
)
eff_example

In [ ]:
grader.check("q10")

In [ ]:
# The sliders below are in the same order as the arguments below,
# in case some of the slider labels are cut off.
interact(effectiveness_calculator,
                 young_vaccinated_prop=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.01),
                 old_vaccinated_prop=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.99),
                 young_risk_vaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.01),
                 young_risk_unvaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.20),
                 old_risk_vaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.10),
                 old_risk_unvaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.50));

In [ ]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_extreme = effectiveness_calculator(**extreme_example())
eff_extreme

In [ ]:
grader.check("q11")

In [ ]:
grader.check_all()


If you were able to go to **Kernel > Restart & Run All** and see all test cases pass above, and you've thoroughly tested your code yourself, you're ready to submit `project.py` to Gradescope!
    